Dataset : Employee data used in lecture linked here.

1.Preprocess Test data and get predictions

2.Compute Mean Absolute Error, Mean Square error for test data.

3.Optional : Read about Ridge and Lasso Regression, implement them and compute evaluation metrics. Do they perform better than Linear Regression?

In [5]:

#Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", 101)
     

In [9]:
data = pd.read_csv('https://raw.githubusercontent.com/abirami1998/NYU-Data-Science-Bootcamp-Spring-2024/main/Week%206/employee.csv')

In [10]:
data

,id,timestamp,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,1,12/11/2018 10:52:26,Slovenia,Full time,Developer,4.783930,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,2,1/5/2017 16:57:50,United States,Full time,DBA,5.000000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,3,12/18/2017 8:13:15,Sweden,Full time,DBA,1.000000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,4,12/27/2018 4:56:52,United States,Full time,DBA,1.000000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,5,12/11/2018 14:07:58,United States,Full time,Developer,3.000000,No,40.0,2.0,Masters,Yes,Yes,7137.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,4273,1/6/2017 14:42:12,United States,Full time,Developer,2.000000,Yes,40.0,0.0,Bachelors (4 years),No,No,4917.000000
4273,4274,12/19/2017 8:59:11,United States,Full time,DBA,4.140793,Yes,NaN,0.0,Bachelors (4 years),No,No,7033.845701
4274,4275,12/24/2018 19:49:32,United States,Full time,Developer,5.000000,No,40.0,0.0,Bachelors (4 years),No,No,7166.000000
4275,4276,12/12/2017 17:12:41,United States,Full time,DBA,4.000000,Yes,40.0,2.0,Bachelors (4 years),Yes,No,9583.000000


In [12]:
data.loc[data['hours_per_week'].isna(), 'hours_per_week'] = data['hours_per_week'].median()
data.loc[data['telecommute_days_per_week'].isna(), 'telecommute_days_per_week'] = data['telecommute_days_per_week'].median()     
data = data.dropna()

In [13]:
data_train = data.copy()
data_train.head()

,id,timestamp,country,employment_status,job_title,job_years,is_manager,hours_per_week,telecommute_days_per_week,education,is_education_computer_related,certifications,salary
0,1,12/11/2018 10:52:26,Slovenia,Full time,Developer,4.78393,Yes,40.0,0.0,Bachelors (4 years),Yes,No,7187.743094
1,2,1/5/2017 16:57:50,United States,Full time,DBA,5.00000,No,40.0,5.0,Bachelors (4 years),Yes,No,10000.000000
2,3,12/18/2017 8:13:15,Sweden,Full time,DBA,1.00000,No,40.0,0.0,Masters,Yes,Yes,7000.000000
3,4,12/27/2018 4:56:52,United States,Full time,DBA,1.00000,No,40.0,2.0,Bachelors (4 years),No,No,8333.000000
4,5,12/11/2018 14:07:58,United States,Full time,Developer,3.00000,No,40.0,2.0,Masters,Yes,Yes,7137.000000


In [14]:
cat_cols = [c for c in data_train.columns if data_train[c].dtype == 'object'
            and c not in ['is_manager', 'certifications']]
cat_data = data_train[cat_cols]
cat_cols
     

['timestamp',
 'country',
 'employment_status',
 'job_title',
 'education',
 'is_education_computer_related']

In [15]:
binary_cols = ['is_manager', 'certifications']
for c in binary_cols:
    data_train[c] = data_train[c].replace(to_replace=['Yes'], value=1)
    data_train[c] = data_train[c].replace(to_replace=['No'], value=0)

In [16]:
final_data = pd.get_dummies(data_train, columns=cat_cols, drop_first= True,dtype=int)


In [18]:

y = final_data['salary']
X = final_data.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print("Training Set Dimensions:", X_train.shape)
print("Validation Set Dimensions:", X_test.shape)
     

Training Set Dimensions: (3408, 4329)
Validation Set Dimensions: (853, 4329)


In [19]:
num_cols = ['job_years','hours_per_week','telecommute_days_per_week']
num_cols

['job_years', 'hours_per_week', 'telecommute_days_per_week']

In [20]:
scaler = StandardScaler()
scaler.fit(X_train[num_cols])
X_train[num_cols] = scaler.transform(X_train[num_cols])

In [21]:
X_train

,id,job_years,is_manager,hours_per_week,telecommute_days_per_week,certifications,timestamp_1/1/2018 13:55:49,timestamp_1/1/2018 22:41:10,timestamp_1/1/2018 9:03:21,timestamp_1/1/2019 20:22:20,timestamp_1/1/2019 22:14:11,timestamp_1/1/2019 5:57:21,timestamp_1/1/2019 7:46:34,timestamp_1/10/2017 0:15:01,timestamp_1/10/2017 0:22:12,timestamp_1/10/2017 0:29:52,timestamp_1/10/2017 0:31:28,timestamp_1/10/2017 0:47:33,timestamp_1/10/2017 10:24:09,timestamp_1/10/2017 10:31:43,timestamp_1/10/2017 10:45:15,timestamp_1/10/2017 10:55:09,timestamp_1/10/2017 11:14:30,timestamp_1/10/2017 11:17:35,timestamp_1/10/2017 11:27:26,timestamp_1/10/2017 11:35:05,timestamp_1/10/2017 11:36:14,timestamp_1/10/2017 11:43:19,timestamp_1/10/2017 11:51:00,timestamp_1/10/2017 11:57:41,timestamp_1/10/2017 12:06:36,timestamp_1/10/2017 12:15:30,timestamp_1/10/2017 12:18:40,timestamp_1/10/2017 12:27:07,timestamp_1/10/2017 12:41:22,timestamp_1/10/2017 12:45:58,timestamp_1/10/2017 12:47:24,timestamp_1/10/2017 12:48:51,timestamp_1/10/2017 13:20:05,timestamp_1/10/2017 13:31:44,timestamp_1/10/2017 13:31:49,timestamp_1/10/2017 13:31:57,timestamp_1/10/2017 13:51:19,timestamp_1/10/2017 14:03:49,timestamp_1/10/2017 14:11:00,timestamp_1/10/2017 14:31:40,timestamp_1/10/2017 14:39:47,timestamp_1/10/2017 14:54:14,timestamp_1/10/2017 14:56:15,timestamp_1/10/2017 15:25:08,...,country_Norway,country_Pakistan,country_Paraguay,country_Peru,country_Philippines,country_Poland,country_Portugal,country_Puerto Rico,country_Romania,country_Russia,country_Saudi Arabia,country_Serbia and Montenegro,country_Singapore,country_Slovakia,country_Slovenia,country_South Africa,country_Spain,country_Sweden,country_Switzerland,country_Syria,country_Taiwan,country_Thailand,country_Turkey,country_Uganda,country_Ukraine,country_United Arab Emirates,country_United Kingdom,country_United States,country_Uruguay,country_Venezuela,country_Vietnam,employment_status_Independent or freelancer or company owner,employment_status_Part time,job_title_Analytics consultant,job_title_Architect,job_title_DBA,job_title_Data Scientist,job_title_Developer,job_title_Engineer,job_title_Manager,job_title_Other,job_title_Principal database engineer,job_title_Sales,job_title_Sr Consultant,education_Bachelors (4 years),education_Doctorate/PhD,education_Masters,education_None (no degree completed),is_education_computer_related_Unknown,is_education_computer_related_Yes
3404,3405,-0.240444,0,-0.484386,-0.629121,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
4009,4010,0.260863,0,-0.484386,-0.629121,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2431,2432,-0.240444,0,-0.484386,-0.629121,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2751,2752,-0.741752,0,-0.484386,-0.629121,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
843,844,0.260863,0,-0.484386,-0.629121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2050,

In [22]:
reg=LinearRegression()
reg.fit(X_train, y_train)

LinearRegression()

In [23]:
mean_squared_error(y_train,reg.predict(X_train))/np.mean(y_train)


1.433178552117801e-21

In [24]:
y_pred = reg.predict(X_test)


In [25]:
mse = mean_squared_error(y_pred, y_test)/np.mean(y_test)
